In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import math
import copy
import pandas as pd
import numpy as np
import itertools
import pickle
import heapq
import time
import json
import os

In [28]:
df_chus = None

item_info = {}

unique_items = []
seq_util_val = []
util_list = []
seq_list = []

seq_no = {}
seq_pos_no = {}
seq_pos_no_util = {}
seq_pos_no_rem_util = {}
seq_pos_no_rbu = {}
seq_pos_no_lru = {}
seq_pos_no_rem_item_count = {}
item_seen_bef = [0] * 1570

item_neighbour = {}

temp_bitmap = []

keys_to_delete = []

item_info_dbv = {}

pattern_av_hash = {}

chus_hash = {}

visited = []
cluster_info = {}
cluster_info_per_reps = {}

dvf={}
pq=[]
pq2=[]
#mu=0
mutl={"mu":0}


In [29]:
def pattern_avoidable(item_l):

  _key_ = sum(dvf[tuple(item_l)]["RemItems"])
  swu_itemset_p = sum(dvf[tuple(item_l)]["SWU"])
  sup_itemset_p = len(dvf[tuple(item_l)]["SID"])


  if(_key_ not in pattern_av_hash):
    pattern_av_hash[_key_] = []

  mk = pattern_av_hash[_key_]
  #print(f"item_l: {item_l}")
  #print(f"mk: {mk})")
  #print()

  if(len(mk)==0):
    pattern_av_hash[_key_].append(item_l)
    #print(f"{item_l} is NEW")
    return False

  else:
    flag = 0
    for itemset_m in range(len(mk)):
      #print(f"itemset_m: {itemset_m} --- len-mk: {len(mk)}")
      #print(f"mk-itemset_m: {mk[itemset_m]}")
      #print()
      if(itemset_m >= len(mk)):
        break
      itemset_p_bar = mk[itemset_m]
      swu_itemset_p_bar = sum(dvf[tuple(itemset_p_bar)]["SWU"])              #item_info_dbv[tuple(itemset_p_bar)]['swu']
      sup_itemset_p_bar = len(dvf[tuple(itemset_p_bar)]["SID"])              #item_info_dbv[tuple(itemset_p_bar)]['support']

      if(swu_itemset_p_bar == swu_itemset_p and
         sup_itemset_p_bar == sup_itemset_p):
        l_p = len(item_l)
        l_p_bar = len(itemset_p_bar)
        if(l_p <= l_p_bar and item_l[l_p-1] == itemset_p_bar[l_p_bar-1]):
          #print(f"{item_l} is BACKWARD-SUB-PATTERN of {itemset_p_bar}")
          return True
        else:
          if(l_p > l_p_bar and item_l[l_p-1] == itemset_p_bar[l_p_bar-1]):
            #print(f"{item_l} is BACKWARD-SUPER-PATTERN of {itemset_p_bar}")
            del pattern_av_hash[_key_][itemset_m]
            flag = 1

    if(flag==1):
      pattern_av_hash[_key_].append(item_l)
      return True

  pattern_av_hash[_key_].append(item_l)
  #print(f"{item_l} is NEW -> last section")
  return False

In [30]:
def raise_mu(k,l,f):
  if(f==1):
    if len(pq) >= k:
      temp_queue = list(pq)
      kth_element = heapq.nsmallest(k, temp_queue)[-1]
      priority, task = kth_element
      ut = -priority
      return ut
  elif(f==2):
    if len(pq2) >= k:
      temp_queue = list(pq2)
      kth_element = heapq.nsmallest(k, temp_queue)[-1]
      priority, task = kth_element
      ut = -priority
      return ut
    else:
      return mutl["mu"] #-----------

In [31]:
def exp_siblings(item_l_child, s_i_temp, i_i_temp, k, l):
  #print("sib exp")
  chus_cand_set_sib = []
  for item_l_child_i in range(len(item_l_child)):
    item_l_child_i_main = item_l_child[item_l_child_i]
    #chus_cand_set_sib += dfs_pattern_ext(item_l_child_i_main, s_i_temp, i_i_temp[item_l_child_i:], k, l)
    dfs_pattern_ext(item_l_child_i_main, s_i_temp, i_i_temp[item_l_child_i:], k, l)
  return

In [32]:
def dfs_pattern_ext(item_l,s_items,i_items,k,l):

  # mu golabl fix
  if(not pattern_avoidable(item_l)):
    if(len(item_l)>=l):
      priority = -sum(dvf[tuple(item_l)]["Utility"])
      key = tuple(item_l)
      heapq.heappush(pq2, (priority, key))
      mutl["mu"] = raise_mu(k,l,2)
    else:
      if(len(item_l)>1):
        priority = -sum(dvf[tuple(item_l)]["Utility"])
        key = tuple(item_l)
        heapq.heappush(pq, (priority, key))
        mutl["mu"] = raise_mu(k,l,1)
      #else:
        #mu = mutl["mu"]


    s_i_temp = []
    i_i_temp = []
    item_l_child = []
    chus_cand_set = []

    for sindx in range(len(s_items)):
      s_i = s_items[sindx]
      key_of_last_item_neighbor = tuple([item_l[-1]])
      neighbor = dvf[key_of_last_item_neighbor]["Neighbors"]

      if((s_i not in neighbor) or (s_i in item_l)):
        continue
      else:
        key_si = tuple([s_i])
        key_iteml = tuple(item_l)

        iteml_sid = dvf[key_iteml]["SID"]
        iteml_event = dvf[key_iteml]["Event"]
        si_sid = dvf[key_si]["SID"]
        si_event = dvf[key_si]["Event"]

        iteml_si_pair = {}
        iteml_rlbu = 0

        for i in range(len(iteml_sid)):
          if(iteml_sid[i] in si_sid):
            si_sid_indx = si_sid.index(iteml_sid[i])
            if(si_event[si_sid_indx] > iteml_event[i]):
              iteml_si_pair_indx = [i, si_sid_indx]
              iteml_si_pair[iteml_sid[i]] = iteml_si_pair_indx
              iteml_rlbu += dvf[key_iteml]["RBU"][i]

        if(iteml_rlbu >= mutl["mu"]):
          s_i_temp.append(s_i)

        sid=[]
        eid=[]
        util = []
        rem_util = []
        swu = []
        rbu = []
        rem_item_count = []
        nbr = set()
        for key, value in iteml_si_pair.items():
          sid.append(key)
          eid.append(dvf[key_si]["Event"][value[1]])
          util.append(dvf[key_iteml]["Utility"][value[0]] + dvf[key_si]["Utility"][value[1]])
          rem_util.append(dvf[key_si]["RemUtil"][value[1]])
          swu.append(dvf[key_iteml]["SWU"][value[0]])
          tmp_rbu = dvf[key_iteml]["Utility"][value[0]] + dvf[key_si]["Utility"][value[1]] + dvf[key_si]["RemUtil"][value[1]]
          rbu.append(tmp_rbu)
          rem_item_count.append(dvf[key_si]["RemItems"][value[1]])

        if(sum(rbu)>=mutl["mu"]):
            iteml_ch = copy.deepcopy(item_l)
            iteml_ch.append(s_i)
            item_l_child.append(iteml_ch)
            dvf[tuple(iteml_ch)] = {
              "SID": sid,
              "Event": eid,
              "Utility": util,
              "RemUtil": rem_util,
              "SWU": swu,
              "RBU": rbu,
              "RemItems": rem_item_count,
              "Neighbors": nbr,
              "Flag": 1,
              "util": sum(util)
            }
            if(len(iteml_ch)>=l):
              priorityn = -sum(dvf[tuple(iteml_ch)]["Utility"])
              keyn = tuple(iteml_ch)
              heapq.heappush(pq2, (priorityn, keyn))
              mutl["mu"] = raise_mu(k,l,2)
            else:
              if(len(iteml_ch)>1):
                priorityn = -sum(dvf[tuple(iteml_ch)]["Utility"])
                keyn = tuple(item_l)
                heapq.heappush(pq, (priorityn, keyn))
                mutl["mu"] = raise_mu(k,l,1)
              #else:
                #mu = mutl["mu"]

    #chus_cand_set += item_l_child
    #chus_cand_set += exp_siblings(item_l_child, s_i_temp, s_i_temp, ms, mu)
    exp_siblings(item_l_child, s_i_temp, s_i_temp, k, l)

    #return chus_cand_set
    return

  else:
    return

In [33]:
def is_sublist(sub_list, super_list):
  """Checks if a sub-list is contained within a super-list."""
  #if sub_list == super_list: return False
  return all(x in super_list for x in sub_list)

In [34]:
def elim_non_closed(cand_set,k):
  chus_set = []
  key_found_bef = {}
  for ik in range(len(cand_set)):
      item_l = cand_set[ik]
      key = sum(dvf[tuple(item_l)]["SID"])
      if(key not in key_found_bef):
        chus_hash[key] = []
        chus_hash[key].append(item_l)
        key_found_bef[key] = 1
      else:
        chus_hash[key].append(item_l)
        key_found_bef[key] += 1

  for key, value in chus_hash.items():
    val1 = copy.deepcopy(value)
    val2 = copy.deepcopy(value)
    for i1 in val1:
      f=0
      for i2 in reversed(val2):
        if((i1 != i2) and (len(dvf[tuple(i1)]["SID"]) == len(dvf[tuple(i2)]["SID"]))):
          if(is_sublist(i1,i2)):
            f=1
            break
          elif(is_sublist(i2,i1)):
            indx = val2.index(i2)
            del val2[indx]
            del val1[indx]

      if(f==0):
        chus_set.append(i1)

  return chus_set[:k]

In [35]:
def TK_CloHusp(k, l):

  filename = '/content/drive/MyDrive/MS Thesis/TK-CHUS-1/Dataset/Mushroom.txt'

  file = open(filename, 'r')

  unique_item_set = set()

  j=0
  total_util = 0
  rem_items = []

  for line in file:
    if(j==2000):break
    row = [int(x) for x in line.replace(":", " ").split()]

    m = math.floor(len(row)/2)
    seq_util_val.append(row[m])
    tran_util = row[m]
    total_util += row[m]
    ul = row[m+1:]
    sl = row[:m]
    temp_set = set(sl)
    unique_item_set.update(temp_set)
    j+=1

    for i in range (0, len(sl)):
      item = sl[i]
      key = tuple([item])
      if key not in dvf:
        dvf[key] = {
          "SID": [],
          "Event": [],
          "Utility": [],
          "RemUtil": [],
          "SWU": [],
          "RBU": [],
          "RemItems": [],
          "Neighbors": set(),
          "Flag": 0,
          "util": 0
        }

        dvf[key]["SID"].append(j)
        dvf[key]["Event"].append(i+1)
        dvf[key]["Utility"].append(ul[i])
        dvf[key]["RemUtil"].append(sum(ul[i+1:]))
        dvf[key]["SWU"].append(tran_util)
        dvf[key]["RBU"].append(sum(ul[i:]))
        dvf[key]["RemItems"].append(len(sl)-(i+1))
        dvf[key]["Neighbors"].update(set(sl[i+1:]))
        dvf[key]["util"] = sum(dvf[key]["Utility"])

        priority = -dvf[key]["util"]  # Negate for max-heap
        heapq.heappush(pq, (priority, key))

      else:
        dvf[key]["SID"].append(j)
        dvf[key]["Event"].append(i+1)
        dvf[key]["Utility"].append(ul[i])
        dvf[key]["RemUtil"].append(sum(ul[i+1:]))
        dvf[key]["SWU"].append(tran_util)
        dvf[key]["RBU"].append(sum(ul[i:]))
        dvf[key]["RemItems"].append(len(sl)-(i+1))
        dvf[key]["Neighbors"].update(set(sl[i+1:]))
        dvf[key]["util"] = sum(dvf[key]["Utility"])

        priority = -dvf[key]["util"]  # Negate for max-heap
        heapq.heappush(pq, (priority, key))

  file.close()

  mutl["mu"] = raise_mu(k,l,1)

  chus_cand = []
  #chus_cand += rem_items

  rem_items = list(unique_item_set)

  for item_i in range(len(rem_items)):
    #chus_cand += dfs_pattern_ext([rem_items[item_i]],rem_items,rem_items[(item_i+1):],ms,mu)
    if(sum(dvf[tuple([rem_items[item_i]])]["SWU"])>=mutl["mu"]):
      dfs_pattern_ext([rem_items[item_i]],rem_items,rem_items[(item_i+1):],k,l)

  pq3 = list(pq2)

  while pq3:
    priority, task = heapq.heappop(pq3)
    chus_cand.append(list(task))
    #print(f"Processing {task} with priority {-priority}")

  chus_set = elim_non_closed(chus_cand,k)

  print()
  print(f"Total no of TK-CHUSPs: {len(chus_set)}")
  print()

  return chus_set

In [36]:
%%time
chus = TK_CloHusp(30,8)


Total no of TK-CHUSPs: 30

CPU times: user 2min 54s, sys: 502 ms, total: 2min 55s
Wall time: 2min 57s


# **CHUSPs**

---



In [37]:
sc=[]
uc=[]
for i in range(len(chus)):
  if(isinstance(chus[i], int)):
    ck = [chus[i]]
  else:
    ck = chus[i]
  sc += [len(dvf[tuple(ck)]["SID"])]
  uc += [sum(dvf[tuple(ck)]["Utility"])]
df_chus = pd.DataFrame({'CHUS Patterns': chus, 'Support': sc, 'Utility': uc})
df_chus

,CHUS Patterns,Support,Utility
0,"[2, 24, 28, 34, 37, 39, 53, 54, 67, 76, 85, 86...",691,171609
1,"[2, 24, 28, 34, 37, 39, 53, 54, 67, 76, 85, 86...",691,171609
2,"[2, 34, 39, 67, 76, 85, 86, 90]",1402,169449
3,"[2, 34, 39, 67, 76, 85, 86, 90]",1402,169449
4,"[23, 34, 59, 63, 85, 86, 90, 93]",1022,158138
5,"[23, 34, 59, 63, 85, 86, 90, 93]",1022,158138
6,"[2, 23, 34, 36, 39, 59, 85, 86, 90, 93]",882,155615
7,"[2, 23, 34, 36, 39, 59, 85, 86, 90, 93]",882,155615
8,"[23, 34, 59, 63, 67, 76, 85, 86, 90, 93]",851,155543
9,"[23, 34, 59, 63, 67, 76, 85, 86, 90, 93]",851,155543


In [ ]:
while pq2:
    priority, task = heapq.heappop(pq2)
    print(f"Processing {task} with priority {-priority}")